In [1]:
import torch
import pandas as pd
import time
from tqdm import tqdm

In [6]:
def samplejd_cosine(path, samplejd_tensor, csvdata, modelname): 
    samplejd_tensor = torch.load(f"{path}{samplejd_tensor}")
    csvdata = pd.read_csv(f"{path}{csvdata}")
    print("불러온 텐서 차원을 확인합니다 : ", samplejd_tensor.shape)

    # 텐서를 5개의 부분으로 나눕니다. ['자격요건', '우대조건', '복지', '회사소개', '주요업무']
    tensor_parts = torch.chunk(samplejd_tensor, chunks=5, dim=1)
    
    # 하나의 행마다 코사인유사도를 계산합니다.
    for index, _ in tqdm(csvdata.iterrows()):

        # '평가대상columns'를 확인해서 one_tensor에 할당합니다.
        match csvdata.at[index, '평가대상columns']:
            case '자격요건':
                one_tensor = tensor_parts[0]

            case '우대조건':
                one_tensor = tensor_parts[1]

            case '복지':
                one_tensor = tensor_parts[2]

            case '회사소개':
                one_tensor = tensor_parts[3]

            case '주요업무':
                one_tensor = tensor_parts[4]

                
        # cosine 유사도를 계산해서 '모델 유사도' 부분을 채웁니다.
        url1 = csvdata.at[index, 'url1_idx']
        url2 = csvdata.at[index, 'url2_idx']
        cos_sim = torch.cosine_similarity(one_tensor[url1],one_tensor[url2],dim=0)
        csvdata.at[index, '모델 유사도'] = cos_sim.item()
    
    #csv파일로 저장합니다.
    csvdata.to_csv(f"{path}samplejd_{modelname}.csv", index=False)

In [7]:
samplejd_cosine(path='../dataset/', # 맨 뒤에 /까지 잘 붙여주기
                samplejd_tensor='multilingual_2.pth', # predict를 통해 얻은 텐서파일
                csvdata='samplejd_ver01.csv', # 코사인유사도 계산 대상 sampleJD. ver01로 사용해주세요!
                modelname='bertmulti_2') # 모델명을 파일명으로 구분하여 csv파일로 저장하게 됩니다.

불러온 텐서 차원을 확인합니다 :  torch.Size([41, 3840])


38it [00:00, 4358.91it/s]


In [4]:
# # (optional) 필요한 경우, 저장된 csv 파일을 불러와서 정렬하여 볼 수 있는 코드입니다. by=['정렬하여 보고 싶은 컬럼명']으로 볼 수 있습니다.
# # url이 너무 길어서 잘려 보인다면 한 데이터씩 출력해서 볼 수 있습니다.

# pd.set_option('display.expand_frame_repr', False)
# sorted_df = pd.read_csv('../dataset/samplejd_bertmulti_ver01.csv').sort_values(by=['url1_idx', 'url2_idx'], ascending=[True, True])